In [1]:
pip install requests beautifulsoup4 pandas flask


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract video details
def get_video_details():
    url = "https://www.youtube.com/@PW-Foundation/videos"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    videos = soup.find_all('a', {'id': 'video-title'}, limit=5)
    
    video_data = []

    for video in videos:
        title = video.get('title')
        video_url = f"https://www.youtube.com{video.get('href')}"
        thumbnail_url = video.find_previous('img')['src']
        views = video.find_next('span', {'class': 'style-scope ytd-video-meta-block'}).text
        posted_time = video.find_next('div', {'id': 'metadata-line'}).find_all('span')[-1].text
        
        video_data.append({
            "Title": title,
            "Video URL": video_url,
            "Thumbnail URL": thumbnail_url,
            "Views": views,
            "Posted Time": posted_time
        })
    
    return video_data

video_details = get_video_details()

# Saving to CSV
df = pd.DataFrame(video_details)
df.to_csv('youtube_videos.csv', index=False)


In [3]:
from flask import Flask, render_template
import pandas as pd

app = Flask(__name__)

@app.route('/')
def home():
    df = pd.read_csv('youtube_videos.csv')
    videos = df.to_dict(orient='records')
    return render_template('index.html', videos=videos)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

D:\Softwares\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
